In [75]:
import pandas as pd
import ollama
from tqdm import tqdm

MODEL = "llama3.2:3b"

In [76]:
# messages = [
#     {"role": "user", "content": "Describe some of the business applications of Generative AI"}
# ]

# response = ollama.chat(model=MODEL, messages=messages)
# print(response['message']['content'])

In [77]:
data = pd.read_csv('./data/HIV.csv')
data_act = data[data['HIV_active'] == 1]
data_inact = data[data['HIV_active'] == 0]
# Sample
data_act = data_act.sample(n=50, random_state=42)
data_inact = data_inact.sample(n=50, random_state=42)
data = pd.concat([data_act, data_inact], ignore_index=True)

input = data['smiles'].values
output = data['HIV_active'].values
print(f'Data length: {len(input)}')

Data length: 100


In [78]:
data['HIV_active'].value_counts()

HIV_active
1    50
0    50
Name: count, dtype: int64

In [79]:
# Zero-shot prediction
zero_shot_preds = []

for smile, label in tqdm(zip(input, output)):
    prompt = f"""You are a chemistry expert. Given the SMILES string below, predict whether the compound is active or inactive against HIV. Answer only "active" or "inactive".

SMILES: {smile}
Prediction:"""

    response = ollama.chat(model=MODEL, messages=[
        {"role": "user", "content": prompt}
    ])

    prediction = response['message']['content'].strip().lower()
    # print(f"SMILES: {smile} | True: {label} | Predicted: {prediction}")
    
    zero_shot_preds.append([1 if prediction == 'active' else 0])

100it [00:23,  4.19it/s]


In [80]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

zero_shot_df = pd.DataFrame(zero_shot_preds, columns=['prediction'])
zero_shot_df['true'] = output

y_true = zero_shot_df['true']
y_pred = zero_shot_df['prediction']

accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

print(f"Zero-shot Accuracy: {accuracy:.4f}")
print(f"Zero-shot Precision: {precision:.4f}")
print(f"Zero-shot Recall: {recall:.4f}")
print(f"Zero-shot F1 Score: {f1:.4f}")

Zero-shot Accuracy: 0.5100
Zero-shot Precision: 0.5333
Zero-shot Recall: 0.1600
Zero-shot F1 Score: 0.2462


In [86]:
# Split into few-shot examples and test input
few_shot_examples = list(zip(input[:50], output[:50]))  # 50 examples
test_set = list(zip(input[50:], output[50:]))

# Build the few-shot context
few_shot_prompt = "You are a chemistry expert. Predict whether each SMILES string represents a compound that is '1' or '0' against HIV. Respond only with 'active' or 'inactive'.\n\n"
for smile, label in tqdm(few_shot_examples):
    few_shot_prompt += f"SMILES: {smile}\nActivity: {label}\n\n"

# Run few-shot predictions
few_shot_preds = []
for test_smile, true_label in tqdm(test_set):
    full_prompt = few_shot_prompt + f"SMILES: {test_smile}\nActivity:"
    
    response = ollama.chat(model=MODEL, messages=[
        {"role": "user", "content": full_prompt}
    ])
    
    prediction = response['message']['content'].strip().lower()
    # print(f"SMILES: {test_smile} | True: {true_label} | Predicted: {prediction}")
    
    few_shot_preds.append([1 if prediction == 'active' else 0])

100%|██████████| 50/50 [00:28<00:00,  1.76it/s]


In [87]:
few_shot_df = pd.DataFrame(few_shot_preds, columns=['prediction'])
few_shot_df['true'] = [label for _, label in test_set]

y_true_fs = few_shot_df['true']
y_pred_fs = few_shot_df['prediction']

accuracy_fs = accuracy_score(y_true_fs, y_pred_fs)
precision_fs = precision_score(y_true_fs, y_pred_fs)
recall_fs = recall_score(y_true_fs, y_pred_fs)
f1_fs = f1_score(y_true_fs, y_pred_fs)

print(f"Few-shot Accuracy: {accuracy_fs:.4f}")
print(f"Few-shot Precision: {precision_fs:.4f}")
print(f"Few-shot Recall: {recall_fs:.4f}")
print(f"Few-shot F1 Score: {f1_fs:.4f}")


Few-shot Accuracy: 0.8200
Few-shot Precision: 0.0000
Few-shot Recall: 0.0000
Few-shot F1 Score: 0.0000


/home/WUR/g0012069/.llms/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
